In [ ]:
import pandas as pd
import numpy as np
import os
from moviepy.editor import *
import json
import pygame
import math

In [ ]:
# krv Быстрый append к DataFrame
def read_group(filename, markup, box, root, key):
    df = []
    for child in markup[box][root]:
        for instance in child['instances']:
            del instance['start']
            del instance['end']
            
            instance['start_ms'] = math.floor(instance['start_ms']/1000)
            instance['end_ms'] = math.floor(instance['end_ms']/1000)
            
            dict = {key : child['key'], 'film' : filename}
            dict.update(instance)
            df.append(dict)
            
    return pd.DataFrame(df)

# Загружаем разметки фильмов с тагами
def load_markup():
    tags = pd.DataFrame()
    persons = pd.DataFrame()
    nudity = pd.DataFrame()

    directory = '.\markup_json'
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            with open(os.path.join(directory, filename)) as json_file:
                markup = json.load(json_file)
                tags = tags.append(read_group(filename, markup, 'tagbox', 'tags', 'tag'))
                persons = persons.append(read_group(filename, markup, 'facebox', 'faces', 'person'))
                nudity = nudity.append(read_group(filename, markup, 'nudebox', 'nudity', 'nude'))

    return tags, persons, nudity

# Загружаем json данные в DataFrame
def load_markup_to_dataframe(tags):
    tags_every_second = pd.DataFrame(columns = ['tag', 'film', 'second'])

    df = []
    for index, row in tags.iterrows():
        for second in range(row.start_ms, row.end_ms + 1):
            df.append({'tag' : row.tag, 'film' : row.film, 'second' : second})

    return fill_second_gaps(
        merge_tags_every_second(
            tags_every_second.append(df)))

# Объединяем все таги каждой минуты
def merge_tags_every_second(df):
    d = []
    tags_every_second = pd.DataFrame(columns = ['film', 'second', 'tags'])
    
    for index, rows in df.groupby(['film', 'second']):
        tags = []
        for idx, val in rows.iterrows():
            tags.append(val[0])
            
        d.append({'film' : index[0], 'second' : index[1], 'tags' : tags})
        
    return pd.DataFrame(d)

# Заполняем пробельные минуты, указывая для них таг None
def fill_second_gaps(tags_every_second):
    d = pd.DataFrame(tags_every_second.sort_values('second')['second'].diff().dropna(), dtype = 'int64')
    d.columns = ['delta']
    for index, row in d[d['delta'] > 1].iterrows():
        idx = tags_every_second.at[index, 'second']
        film = tags_every_second.at[index, 'film']
        delta = row.delta
        for second in range(idx - delta + 1, idx):
            tags_every_second = tags_every_second.append({'film' : film, 'second' : second,
                                                          'tags' : ['None']}, ignore_index = True)
    
    return tags_every_second.sort_values('second')

# Создаем сцены , объединяя минуты похожие более чем threshold
def build_scenes(tags_every_second, threshold):
    scenes = []
    change = [0]
    prev = []
    prev_second = -1
    union = []
    counter = 1
    duration = 1
    start = -1

    for index, row in tags_every_second.iterrows():
        if start == -1:
            start = row.second
            prev = [*row.tags]
        else:
            union = set.union(set(prev), set([*row.tags]))
            intersect = set.intersection(set(prev), set([*row.tags]))
            similarity = len(intersect)/len(union)
            if similarity > threshold:
                prev = union
                duration += 1
            else:
                if list(prev) == ['None']:
                    title = 'Ничего не удалось распознать'
                else:
                    title = 'Scene_' + str(counter)
                    counter += 1
                    
                scenes.append({'title' : title, 'start' : start, 'duration' : duration, 
                               'tags' : list(prev)})
                start = row.second
                duration = 1
                prev = [*row.tags]
                
            if scene_debug_break < counter:
                break
         
    if list(prev) == ['None']:
        title = 'Ничего не удалось распознать'
    else:
        title = 'Scene_' + str(counter)
        
    scenes.append({'title' : title, 'start' : start, 'duration' : duration, 'tags' : list(prev)})
     
    return pd.DataFrame(scenes)

In [ ]:
scene_debug_break = 2000000
tags, persons, nudity = load_markup()
tags_every_second = load_markup_to_dataframe(tags)
scenes = build_scenes(tags_every_second, threshold = 0.05)

In [ ]:
new_scenes = scenes.copy()
working_index = -1
previous_none_index = -1

for index, row in new_scenes[new_scenes.title == 'Ничего не удалось распознать'].iterrows():
    if index == len(new_scenes) - 1:
        break

    if previous_none_index == -1:
        previous_none_index = index
       
    if index - previous_none_index != 2:
        working_index = index - 1
        
    tags1 = new_scenes.at[working_index, 'tags']
    tags2 = new_scenes.at[index + 1, 'tags']

    union = set.union(set(tags1), set(tags2))
    intersect = set.intersection(set(tags1), set(tags2))
    similarity = len(intersect)/len(union)
    if similarity > 0.05:
        new_scenes.at[working_index, 'duration']  =  new_scenes.at[working_index, 'duration'] + new_scenes.at[index, 'duration'] + new_scenes.at[index + 1, 'duration']
        new_scenes.at[index + 1, 'tags'] = ['Need to delete']
        new_scenes.at[index + 1, 'duration'] = -1
        new_scenes.at[index, 'title'] = 'Удалить'
        new_scenes.at[working_index, 'tags'] = list(union)
    else:
        previous_none_index = -1
        
new_scenes = new_scenes[(new_scenes.title != 'Удалить')&(new_scenes.duration != -1)]


In [ ]:
clip_list = []

for index, scene in new_scenes.iterrows():
    txt_clip = TextClip(txt = scene['title'], fontsize = 70, color = 'white').set_duration(scene['duration'])
    clip_list.append(txt_clip)
    final_clip = concatenate(clip_list, method = "compose")
    
myvideo = VideoFileClip('videos/Брат.mp4')#.subclip(1, 191)
final = CompositeVideoClip([myvideo, final_clip])
filename = "videos/Брат_with_subtitles_subclip.mp4"
final.write_videofile(filename)

In [ ]:
new_scenes.style.set_properties(subset=['tags'], **{'width': '500px'})